In [152]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import numpy as np
import pandas as pd
import random
import cStringIO
import time
import os




In [157]:
categories = ['Title','Image','Map', 'Video', 'Link', 'Paragraph', 'Button']
values = [0, 1, 2, 3, 4, 5, 6]
dictionary = dict(zip(values,categories))

df = pd.read_excel('testfile.xlsx')
X_train, X_test, y_train, y_test = train_test_split(df.drop('Label', 1), df['Label'], test_size=0.2,
                                                        random_state=42)

#DataTest
documents = [s.encode('utf-8').strip() for s in X_train['Sentence']]

#Training Set
document = [s.encode('utf-8').strip() for s in X_test['Sentence']]


In [158]:
#Like a list of Documents to apply tfidf
docs_data = [cStringIO.StringIO(x) for x in documents]

#CountVectorizer
count_vect = CountVectorizer(input="file")
X_train_counts = count_vect.fit_transform(docs_data)

#TFIDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)



In [159]:
#SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5,epsilon=0.1, random_state=42))
#Linear SVC and Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', LinearSVC(loss='squared_hinge',dual=False, tol=1e-3)),])
    
# text_clf.fit(documents, y_train)
# predicted = text_clf.predict(document)

#GridSearch
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf.fit(documents, y_train)
predicted = gs_clf.predict(document)

print (np.mean(predicted == y_test))
print(predicted)
#predicted = [dictionary[k] for k in predicted]





0.88
[3 0 0 6 1 3 0 2 3 2 3 1 0 1 5 1 0 3 2 2 0 5 0 3 3 2 4 4 4 1 0 2 1 1 6 2 1
 0 5 5 3 6 0 4 2 2 1 1 0 1 4 0 6 3 3 1 1 0 1 1 2 3 1 1 3 1 4 3 3 1 2 3 6 6
 1]
